In [9]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
import numpy as np

digits = datasets.load_digits()
X = digits.images.astype(np.float32)
y = digits.target.astype(np.int32)

# Split it into train / test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Split X_train again to create validation data
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2)

In [10]:
%%time
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 8x8 pixels
    #   we define the last three elements as 8x8x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    X = tf.reshape(X, [-1, 8, 8, 1])
    print(X)
    
    # now lets define the convolution
    # here we are saying to find 24 different filter outputs from the image
    # using a 4x4 weight kernel
    features = layers.conv2d(inputs=X, 
                             num_outputs=24, 
                             kernel_size=[3, 3])
    print(features)
    
    # take the max of each of the 24 filter outputs (lose spatial resolution)
    features = tf.reduce_max(input_tensor=features, 
                             reduction_indices=[1, 2])
    
    print(features)
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    print('===============================')
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=1000, 
                                       learning_rate=0.05, batch_size=128)

classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 24), dtype=float32)
Tensor("Max:0", shape=(?, 24), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 24), dtype=float32)
Tensor("Max:0", shape=(?, 24), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.922222222222
CPU times: user 54.8 s, sys: 1.63 s, total: 56.5 s
Wall time: 18.6 s


In [11]:
%%time
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 8x8 pixels
    #   we define the last three elements as 8x8x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    img_wh = 8 # height and width of images
    X = tf.reshape(X, [-1, img_wh, img_wh, 1])
    print(X)
    
    # now lets define the convolution
    # here we are saying to find n_out different filters from the image
    # using 4x4 weight kernels
    n_out = 12
    features = layers.conv2d(inputs=X, 
                             num_outputs=n_out, 
                             kernel_size=[3, 3])
    print(features)
    
    # pool the outputs of the filters 
    # stride controls downsampling weight
    kernel = [1, 2, 2, 1]
    stride = [1, 2, 2, 1]
    features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
    print(features)
    
    # make the weights a column vector
    n_rows = int(img_wh/stride[1])
    n_cols = int(img_wh/stride[2])
    features = tf.reshape(features,[-1, n_out*n_rows*n_cols])
    print(features)
    
    # then make a fully connected layer with sigmoid nonlinearity 
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    print('===============================')
    
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=1500, 
                                       learning_rate=0.05, batch_size=128)

classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 12), dtype=float32)
Tensor("MaxPool:0", shape=(?, 4, 4, 12), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 192), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 12), dtype=float32)
Tensor("MaxPool:0", shape=(?, 4, 4, 12), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 192), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.986111111111
CPU times: user 15.9 s, sys: 2.48 s, total: 18.3 s
Wall time: 9.26 s


In [12]:
%%time
# make it deeper
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 8x8 pixels
    #   we define the last three elements as 8x8x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    img_wh = 8 # height and width of images
    features = tf.reshape(X, [-1, img_wh, img_wh, 1])
    print(X)
    
    nlayers = 2
    n_out = 24
    for _ in range(nlayers):
        # now lets define the convolution
        # here we are saying to find n_out different filters from the image
        # using 4x4 weight kernels
        
        features = layers.conv2d(inputs=features, 
                                 num_outputs=n_out, 
                                 kernel_size=[3, 3])
        print(features)

        # pool the outputs of the filters 
        # stride controls downsampling weight
        kernel = [1, 2, 2, 1]
        stride = [1, 2, 2, 1]
        features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
        print(features)
    
    # make the weights a column vector
    n_rows = int(img_wh/(stride[1]*nlayers))
    n_cols = int(img_wh/(stride[2]*nlayers))
    features = tf.reshape(features,[-1, n_out*n_rows*n_cols])
    print(features)
    
    # then make a fully connected layer with sigmoid nonlinearity 
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    print('===============================')
    
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=2000, 
                                       learning_rate=0.05, batch_size=256)

classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 24), dtype=float32)
Tensor("MaxPool:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 2, 2, 24), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 96), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 8, 8, 24), dtype=float32)
Tensor("MaxPool:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 2, 2, 24), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 96), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.988888888889
CPU times: user 1min 20s, sys: 12.1 s, 

In [35]:
%%time
# LeNet example (dumbed down to )
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 8x8 pixels
    #   we define the last three elements as 8x8x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    img_wh = 8 # height and width of images
    features = tf.reshape(X, [-1, img_wh, img_wh, 1])
    print(features)
    
    nlayers = 1
    n_out = 24
    
    features = layers.conv2d(inputs=features, 
                            num_outputs=n_out, 
                            kernel_size=[3, 3])
    
    kernel = [1, 2, 2, 1]
    stride = [1, 2, 2, 1]
    features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
    
    n_out = 24
    for _ in range(nlayers):
        # now lets define the convolution
        # here we are saying to find n_out different filters from the image
        # using 4x4 weight kernels
        
        features = layers.conv2d(inputs=features, 
                                 num_outputs=n_out, 
                                 kernel_size=[2, 2])
        print(features)

        # pool the outputs of the filters 
        # no downsampling
        kernel = [1, 2, 2, 1]
        stride = [1, 2, 2, 1]
        features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
        print(features)
    
    # make the weights a column vector
    features = layers.flatten(features)
    print(features)
    
    features = tf.nn.relu(features)
    features = layers.stack(features, layers.fully_connected, [100])
    features = tf.nn.relu(features)
    features = layers.stack(features, layers.fully_connected, [100])
    features = tf.nn.relu(features)
    #features = tf.nn.sigmoid(features)
    print(features)
    
    # then make a fully connected layer with sigmoid nonlinearity 
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    
    print('===============================')
    
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=5000, 
                                       learning_rate=0.05, batch_size=128)

classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 2, 2, 24), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 96), dtype=float32)
Tensor("Relu_2:0", shape=(?, 100), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 8, 8), dtype=float32)
Tensor("Reshape:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 4, 4, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 2, 2, 24), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 96), dtype=float32)
Tensor("Relu_2:0", shape=(?, 100), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.997222222222
CPU times: user 1min 37s, sys: 14.8 s, total: 1min 52s
Wall time: 50.7 s


In [1]:
# more data for handwriting recognition?
# Let's use Raschka's implementation for using the mnist dataset:
# https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch12/ch12.ipynb
import os
import struct
import numpy as np
# from sklearn.preprocessing import RobustScaler
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images.astype(np.float32), labels.reshape((-1,1)).astype(np.int32)

X_train, y_train = load_mnist('data/', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

X_test, y_test = load_mnist('data/', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

X_train = X_train/255.0
X_test = X_test/255.0

y_train.shape

Rows: 60000, columns: 784
Rows: 10000, columns: 784


(60000, 1)

In [10]:
%%time

import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

from sklearn.metrics import accuracy_score

feature_columns = learn.infer_real_valued_columns_from_input(X_train)
clf = learn.DNNClassifier(feature_columns=feature_columns, 
                          activation_fn=layers.nn.relu,
                          hidden_units=[100,50,50], 
                          n_classes=10)

clf.fit(X_train, y_train, batch_size=256, max_steps=5000)

yhat = list(clf.predict(X_test, as_iterable=True))

print(accuracy_score(yhat,y_test))

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
Instructions for updating:
Please use tf.contrib.framework.load_variable instead


0.975
CPU times: user 1min 31s, sys: 5.1 s, total: 1min 36s
Wall time: 38.7 s


In [16]:
%%time
# LeNet example (dumbed down to )
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 28x28 pixels
    #   we define the last three elements as 28x28x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    img_wh = 28 # height and width of images
    features = tf.reshape(X, [-1, img_wh, img_wh, 1])
    print(features)
    
    nlayers = 1
    n_out = 24
    
    features = layers.conv2d(inputs=features, 
                            num_outputs=n_out, 
                            kernel_size=[3, 3])
    
    kernel = [1, 2, 2, 1]
    stride = [1, 2, 2, 1]
    features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
    
    n_out = 24
    for _ in range(nlayers):
        # now lets define the convolution
        # here we are saying to find n_out different filters from the image
        # using 4x4 weight kernels
        
        features = layers.conv2d(inputs=features, 
                                 num_outputs=n_out, 
                                 kernel_size=[2, 2])
        print(features)

        # pool the outputs of the filters 
        # no downsampling
        kernel = [1, 2, 2, 1]
        stride = [1, 2, 2, 1]
        features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
        print(features)
    
    # make the weights a column vector
    features = layers.flatten(features)
    print(features)
    
    features = tf.nn.relu(features)
    features = layers.stack(features, layers.fully_connected, [100])
    features = tf.nn.relu(features)
    features = layers.stack(features, layers.fully_connected, [100])
    features = tf.nn.relu(features)
    #features = tf.nn.sigmoid(features)
    print(features)
    
    # then make a fully connected layer with sigmoid nonlinearity 
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    
    print('===============================')
    
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=2000, 
                                       learning_rate=0.05, batch_size=128)

classifier.fit(X_train, y_train)
score = accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 784), dtype=float32)
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 14, 14, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 24), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 1176), dtype=float32)
Tensor("Relu_2:0", shape=(?, 100), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 784), dtype=float32)
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 14, 14, 24), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 24), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 1176), dtype=float32)
Tensor("Relu_2:0", shape=(?, 100), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.9865
CPU times: user 6min 44s, sys: 47.4 s, total: 7min 32s
Wall time: 3min 22s


In [17]:
%%time
# Copy TensorFlow Architecture from 
#   Deep MNIST for experts
#   https://www.tensorflow.org/versions/r0.11/tutorials/mnist/pros/index.html

import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

def conv_model(X, y):
    print('===============================')
    print(X)
    # get in format expected by conv2d
    # (batch_size,width, height,color_channels)
    # since our images are gray scale and 28x28 pixels
    #   we define the last three elements as 28x28x1
    # we don't know the batch size, so just let it 
    # figure that out from the input data (-1 designation)
    img_wh = 28 # height and width of images
    features = tf.reshape(X, [-1, img_wh, img_wh, 1])
    print(features)
    
    # create 32 filters of 5x5 size 
    n_out = 32
    kernel = [5,5]
    features = layers.conv2d(inputs=features, 
                            num_outputs=n_out, 
                            kernel_size=kernel)
    
    # add a bias and pass through relu (for concentrated gradients)
    features = tf.nn.relu(layers.bias_add(features))
    
    # 2x2 max pool to reduce image size to 14x14
    kernel = [1, 2, 2, 1]
    stride = [1, 2, 2, 1]
    features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
    
    # create 64 filters of 5x5 size
    n_out = 64
    kernel = [5,5]
    features = layers.conv2d(inputs=features, 
                            num_outputs=n_out, 
                            kernel_size=kernel)
    
    # add a bias and pass through relu (for concentrated gradients)
    features = tf.nn.relu(layers.bias_add(features))
    
    # 2x2 max pool to reduce image size to 7x7
    kernel = [1, 2, 2, 1]
    stride = [1, 2, 2, 1]
    features = tf.nn.max_pool(features, ksize=kernel,strides=stride, padding='SAME')
    

    # make the weights a column vector, 7x7x64 = 3136
    features = layers.flatten(features)
    print(features)
    
    # pass through fully connected layer with 1024 hidden neurons, W=3136x1024
    features = layers.stack(features, layers.fully_connected, [1024])
    
    # add bias and pass through relu
    features = tf.nn.relu(layers.bias_add(features))
    print(features)
    
    # then make a fully connected layer with bias and sigmoid nonlinearity 
    #  which... is... just logistic regression with one versus all
    pred, loss = learn.models.logistic_regression(features, y)
    print(pred)
    
    print('===============================')
    
    return pred, loss


# Create a classifier, train and predict.
classifier = learn.TensorFlowEstimator(model_fn=conv_model, 
                                       n_classes=10, steps=20000, 
                                       learning_rate=0.05, batch_size=64)

# this operation can take a little while to complete
#   Google says it should take about 30 minutes, but 
#   my machine took a lot longer...
classifier.fit(X_train, y_train)

# now predict the outcome
score = accuracy_score(y_test, classifier.predict(X_test))
print(score)

Tensor("input:0", shape=(?, 784), dtype=float32)
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 3136), dtype=float32)
Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
Tensor("input:0", shape=(?, 784), dtype=float32)
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(?, 3136), dtype=float32)
Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)
Tensor("logistic_regression/softmax_classifier/Softmax:0", shape=(?, 10), dtype=float32)
0.9926
CPU times: user 4h 1min 13s, sys: 31min 8s, total: 4h 32min 21s
Wall time: 1h 31min 33s
